In [1]:
# until ttk is installed, add parent dir to path
import sys
sys.path.insert(0, '..')

In [2]:
# typicaL imports
import pandas as pd
import numpy as np
import re

import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

import matplotlib.pyplot as plt

import spacy
import ete3
import seaborn

from ttk.corpus import load_headline_corpus

In [3]:
%%time
# load the corpus
corpus = load_headline_corpus(verbose=True)

print ('Headlines:', len(corpus.sents()))

Loading corpus from: S:\git\tacticsiege\tactictoolkit\ttk\..\env\corpus\dated\2017_08_22\corpus
Corpus loaded.
Headlines: 190447
Wall time: 10.9 s


In [6]:
from ttk.preprocessing import Seq2WordVecTransformer
from ttk.preprocessing import SeqPaddingTransformer

from ttk.corpus import CategorizedDatedCorpusReporter

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from ttk.preprocessing import Seq2IndexTransformer

label_enc = LabelEncoder()
reporter = CategorizedDatedCorpusReporter()
corpus_df = reporter.to_data_frame(corpus, categories=['Washington Post', 'CNN'])

X = corpus_df['content'].values
X = [s.split() for s in X]
y = label_enc.fit_transform(corpus_df['category'].values)


def token_filter(t, s):
    return t.isalpha()

def token_processor(t, s):
    return t.lower()

indexer = Seq2IndexTransformer(add_delimiters=False, token_mapping_func=token_processor, token_filter_func=token_filter)
pad = SeqPaddingTransformer()

In [7]:
X_indexed = indexer.fit_transform(X)
for i in range(5):
    print ('X[%i]:' % i, X_indexed[i])

Max index: 17565
X[0]: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
X[1]: [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
X[2]: [31, 32, 33, 34, 35, 36, 21, 37, 38, 39, 20, 24, 25, 40, 39, 27]
X[3]: [41, 42, 27, 24, 25, 26, 7, 43, 44]
X[4]: [33, 45, 46, 14, 15, 16, 47, 48, 49, 50, 21, 51, 52]


In [8]:
X_padded = pad.fit_transform(X_indexed)
for i in range(5):
    print ('X[%i]' % i, X_padded[i])

X[0] [  2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.  16.
  17.  18.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
X[1] [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.  30.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
X[2] [ 31.  32.  33.  34.  35.  36.  21.  37.  38.  39.  20.  24.  25.  40.  39.
  27.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
X[3] [ 41.  42.  27.  24.  25.  26.   7.  43.  44.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
X[4] [ 33.  45.  46.  14.  15.  16.  47.  48.  49.  50.  21.  51.  52.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [9]:
# split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.3, random_state=0)
print ('%i Training examples, %i Testing examples, Shape: %s' % (len(X_train), len(X_test), X_train.shape))

24229 Training examples, 10384 Testing examples, Shape: (24229, 27)


In [13]:
%%time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error

N, t = X_train.shape
X_train_s = X_train.reshape(N, t, 1)

model = Sequential()
model.add(LSTM(4, input_shape=(27, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train_s, y_train, epochs=100, batch_size=1, verbose=2)
print ("Done fitting model.")

Epoch 1/100
634s - loss: 0.1979
Epoch 2/100
589s - loss: 0.1936
Epoch 3/100
577s - loss: 0.1927
Epoch 4/100
575s - loss: 0.1927
Epoch 5/100
575s - loss: 0.1924
Epoch 6/100
577s - loss: 0.1922
Epoch 7/100
579s - loss: 0.1921
Epoch 8/100
576s - loss: 0.1923
Epoch 9/100
579s - loss: 0.1920
Epoch 10/100
579s - loss: 0.1925
Epoch 11/100
579s - loss: 0.1921
Epoch 12/100
580s - loss: 0.1919
Epoch 13/100
580s - loss: 0.1916
Epoch 14/100
582s - loss: 0.1918
Epoch 15/100
579s - loss: 0.1918
Epoch 16/100
580s - loss: 0.1916
Epoch 17/100
579s - loss: 0.1916
Epoch 18/100
576s - loss: 0.1918
Epoch 19/100
577s - loss: 0.1917
Epoch 20/100
579s - loss: 0.1915
Epoch 21/100
578s - loss: 0.1915
Epoch 22/100


KeyboardInterrupt: 

In [ ]:
# make predictions
trainPredict = model.predict(X_train)
test